In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [ ]:
path = '/axp/rim/imsadsml/warehouse/sagra39/Kaggle/porto/'
train_data = pd.read_csv(path + 'train_data.csv')
test_data = pd.read_csv(path + 'test_data.csv')
print (train_data.shape, test_data.shape)

In [33]:
train_data.head(2)

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0


## Data Split

In [3]:
from sklearn.model_selection import train_test_split
final_data1 = train_data
train_data_x = final_data1.drop(['id', 'target'], axis=1)
train_data_y = final_data1['target']
train_x, valid_x, train_y, valid_y = train_test_split(train_data_x, train_data_y, test_size=0.20, random_state=42)
print (train_x.shape, valid_x.shape, train_y.shape, valid_y.shape)

((476169, 57), (119043, 57), (476169,), (119043,))


## XGBoost CV

In [19]:
# clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
#                    cv=StratifiedKFold(n_splits=3), 
#                    scoring='roc_auc',
#                    verbose=2, refit=True)

# clf.fit(train_x, train_y)

In [ ]:
gbm = xgb.XGBClassifier()
gbm_params = {
    'learning_rate': [0.01, 0.005],
    'n_estimators': [1000],
    'max_depth': [6,8],
    'subsample':[0.8],
    'colsample_bytree':[0.8],
    'objective':['binary:logistic'],
    'min_child_weight':[100],
    'seed':[1008],
    'nthread':[4]
}
cv = StratifiedKFold(n_splits=4)
grid = GridSearchCV(gbm, gbm_params,scoring='roc_auc',cv=cv,verbose=10,n_jobs=-1)
grid.fit(train_data_x, train_data_y)

print (grid.best_params_)

In [10]:
train_matrix = xgb.DMatrix(data=train_x, label=train_y)
valid_matrix = xgb.DMatrix(data=valid_x, label=valid_y)
test_matrix = xgb.DMatrix(data=test_x, label=test_y)

In [13]:
pred_valid_xgb = pd.DataFrame(model_xgb.predict(valid_matrix), columns=['score'])
pred_test_xgb  = pd.DataFrame(model_xgb.predict(test_matrix), columns=['score'])

NameError: name 'model_xgb' is not defined

## XGBoost Features

In [18]:
param = {}
param['booster'] = 'gbtree'
param['objective'] = 'rank:pairwise'
param['eta'] = 0.10
param['seed']=  1008
param['max_depth'] = 3 
param['min_child_weight'] = 1000
param['silent'] =  1  
param['nthread'] = 12
param['subsample'] = 0.5
param['gamma'] = 1.0
param['colsample_bytree'] = 1.0
param['colsample_bylevel'] = 1.0                  

In [19]:
train_matrix = xgb.DMatrix(data=train_x, label=train_y)
valid_matrix = xgb.DMatrix(data=valid_x, label=valid_y)
test_matrix = xgb.DMatrix(data=test_x, label=test_y)
model_xgb_features = xgb.train(params=param, dtrain=train_matrix, num_boost_round=25)

In [21]:
prob_train = pd.DataFrame(model_xgb_features.predict(train_matrix), columns=['model_prob'])
prob_valid = pd.DataFrame(model_xgb_features.predict(valid_matrix), columns=['model_prob'])
prob_test  = pd.DataFrame(model_xgb_features.predict(test_matrix), columns=['model_prob'])

In [24]:
columns = ['bst' + str(x+1) for x in range(25)]
pred_train = pd.DataFrame(model_xgb_features.predict(train_matrix, pred_leaf=True), columns=columns)
pred_valid = pd.DataFrame(model_xgb_features.predict(valid_matrix, pred_leaf=True), columns=columns)
pred_test  = pd.DataFrame(model_xgb_features.predict(test_matrix, pred_leaf=True), columns=columns)
print(pred_train.shape, pred_valid.shape, pred_test.shape)

(416648, 25) (119638, 25) (58926, 25)


In [27]:
train = train.join(prob_train).join(pred_train)
valid = valid.join(prob_train).join(pred_train)
test = test.join(prob_train).join(pred_train)

## XGBoost Model

In [4]:
parameters = {}
parameters['learning_rate']   = 0.01
parameters['max_depth']        = 8
parameters['subsample']        = 0.8
parameters['colsample_bytree'] = 0.8
parameters['objective']        = 'binary:logistic'
parameters['seed']             = 1008
parameters['nthread']          = 4
parameters['min_child_weight']      =100

In [5]:
train_matrix = xgb.DMatrix(data=train_data_x, label=train_data_y)
# valid_matrix = xgb.DMatrix(data=valid_x, label=valid_y)
# test_matrix = xgb.DMatrix(data=test_x, label=test_y)
model_xgb = xgb.train(params=parameters, dtrain=train_matrix, num_boost_round=1000)

In [12]:
valid_matrix = xgb.DMatrix(data=valid_x, label=valid_y)

In [13]:
pred_valid_xgb = pd.DataFrame(model_xgb.predict(valid_matrix), columns=['score'])

In [14]:
pred_valid_xgb.columns=['1_score']
pred_test_xgb.columns=['1_score']
print(pred_valid_xgb.shape, pred_test_xgb.shape)

(119638, 1) (58926, 1)


In [15]:
from sklearn.metrics import roc_curve, auc
fpr_valid,tpr_valid,thresholds_valid = roc_curve(valid_y,pred_valid_xgb['1_score'])
fpr_test,tpr_test,thresholds_test = roc_curve(test_y,pred_test_xgb['1_score'])

print ('valid:', 2*auc(fpr_valid,tpr_valid) - 1, 'test:', 2*auc(fpr_test,tpr_test) - 1)

valid: 0.273020303474 test: 0.280466653652


In [20]:
# final_x = test_data.drop('id', axis=1)
# final_matrix = xgb.DMatrix(data=final_x)
# pred_final =model_xgb.predict(final_matrix)
# dt_final=pd.DataFrame(pred_final)
dt_final.columns=['target']

In [19]:
dt_final.head()

,0
0,0.027547
1,0.028233
2,0.025350
3,0.015057
4,0.035876


In [21]:
path_new = '/axp/rim/imsadsml/warehouse/sagra39/Kaggle/porto/'
final_sub = test_data[['id']].join(dt_final['target'])
final_sub.to_csv(path_new + 'final_sub4.csv', index=False)

## LightGBM model

In [30]:
model=lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.005, n_estimators=3000, 
                         max_bin=255, subsample_for_bin=50000, objective=None, min_split_gain=0.0, min_child_weight=5,
                         min_child_samples=10, subsample=1.0, subsample_freq=1, colsample_bytree=1.0, reg_alpha=0.0, 
                         reg_lambda=0.0, silent=True)

model.fit(train_x,train_y,
        eval_set=[(valid_x,valid_y)],
        eval_metric='logloss',
        early_stopping_rounds=30)

[1]	valid_0's binary_logloss: 0.688872
Training until validation scores don't improve for 30 rounds.
[2]	valid_0's binary_logloss: 0.68464
[3]	valid_0's binary_logloss: 0.68045
[4]	valid_0's binary_logloss: 0.6763
[5]	valid_0's binary_logloss: 0.672197
[6]	valid_0's binary_logloss: 0.668128
[7]	valid_0's binary_logloss: 0.664105
[8]	valid_0's binary_logloss: 0.660116
[9]	valid_0's binary_logloss: 0.65617
[10]	valid_0's binary_logloss: 0.652256
[11]	valid_0's binary_logloss: 0.648386
[12]	valid_0's binary_logloss: 0.644548
[13]	valid_0's binary_logloss: 0.640747
[14]	valid_0's binary_logloss: 0.636985
[15]	valid_0's binary_logloss: 0.633256
[16]	valid_0's binary_logloss: 0.629565
[17]	valid_0's binary_logloss: 0.625905
[18]	valid_0's binary_logloss: 0.622266
[19]	valid_0's binary_logloss: 0.618679
[20]	valid_0's binary_logloss: 0.615107
[21]	valid_0's binary_logloss: 0.611585
[22]	valid_0's binary_logloss: 0.608079
[23]	valid_0's binary_logloss: 0.604618
[24]	valid_0's binary_logloss: 0

LGBMClassifier(boosting_type='gbdt', colsample_bytree=1.0,
        learning_rate=0.005, max_bin=255, max_depth=-1,
        min_child_samples=10, min_child_weight=5, min_split_gain=0.0,
        n_estimators=3000, nthread=-1, num_leaves=31, objective='binary',
        reg_alpha=0.0, reg_lambda=0.0, seed=0, silent=True, subsample=1.0,
        subsample_for_bin=50000, subsample_freq=1)

In [31]:
pred_valid =model.predict_proba(valid_x)
pred_test =model.predict_proba(test_x)

dt_valid=pd.DataFrame(pred_valid)
dt_test=pd.DataFrame(pred_test)

dt_valid.columns=['0_score','1_score']
dt_test.columns=['0_score','1_score']
print(dt_valid.shape, dt_test.shape)

(119638, 2) (58926, 2)


In [32]:
from sklearn.metrics import roc_curve, auc
fpr_valid,tpr_valid,thresholds_valid = roc_curve(valid_y,dt_valid['1_score'])
fpr_test,tpr_test,thresholds_test = roc_curve(test_y,dt_test['1_score'])

print ('valid:', 2*auc(fpr_valid,tpr_valid) - 1, 'test:', 2*auc(fpr_test,tpr_test) - 1)

valid: 0.0645331064403 test: 0.0517330951421


## Grid Search

In [121]:
estimator = lgb.LGBMRegressor(num_leaves=31)

# model=lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.001, n_estimators=3000, 
#                          max_bin=255, subsample_for_bin=50000, objective=None, min_split_gain=0.0, min_child_weight=5,
#                          min_child_samples=10, subsample=1.0, subsample_freq=1, colsample_bytree=1.0, reg_alpha=0.0, 
#                          reg_lambda=0.0, silent=True)

param_grid = {
    'boosting_type' : ['gbdt'],
    'learning_rate': [0.005],
    'n_estimators': [1000, 500]
}

gbm = GridSearchCV(estimator, param_grid)

gbm.fit(train_x,train_y)

print('Best parameters found by grid search are:', gbm.best_params_)

Best parameters found by grid search are: {'boosting_type': 'gbdt', 'learning_rate': 0.005, 'n_estimators': 1000}


In [122]:
pred_valid =gbm.predict(valid_x)
pred_test =model.predict(test_x)

dt_valid=pd.DataFrame(pred_valid)
dt_test=pd.DataFrame(pred_test)

dt_valid.columns=['1_score']
dt_test.columns=['1_score']
print(dt_valid.shape, dt_test.shape)

(119638, 1) (58926, 1)


In [123]:
from sklearn.metrics import roc_curve, auc
fpr_valid,tpr_valid,thresholds_valid = roc_curve(valid_y,dt_valid['1_score'])
fpr_test,tpr_test,thresholds_test = roc_curve(test_y,dt_test['1_score'])

print ('valid:', 2*auc(fpr_valid,tpr_valid) - 1, 'test:', 2*auc(fpr_test,tpr_test) - 1)

valid: 0.283041165043 test: 0.000931098696462


In [118]:
dt_valid.head()

,0
0,0.041768
1,0.083140
2,0.038555
3,0.070824
4,0.043463


## Test data Submission

In [17]:
final_x = test_data.drop('id', axis=1)
pred_final =model.predict_proba(final_x)
dt_final=pd.DataFrame(pred_final)
dt_final.columns=['0_score','target']

NameError: name 'model' is not defined

In [76]:
final_sub.head()

,id,target
0,0,0.028596
1,1,0.027498
2,2,0.028583
3,3,0.018398
4,4,0.038879


## Ensemble

In [ ]:
path_new = '/axp/rim/imsadsml/warehouse/sagra39/Kaggle/porto/'
CNN = pd.read_csv(path_new + 'CNN.csv')
sub8 = pd.read_csv(path_new + 'final_sub8.csv')
# dt_final.columns = ['target']

In [ ]:
Ensemble_cnn_sub8 = sub8.merge(CNN, on='id', how='inner')